# Hướng dẫn truy vấn dữ liệu thị giác dùng fiftyone

Đây là hướng dẫn dùng cho các đội tham dự AI Challenge 2022. Hướng dẫn này nhầm mục đích giới thiệu cho các đội một phương pháp cơ bản để truy vấn dữ liệu dựa trên thông tin BTC cung cấp và giới thiệu công cụ fiftyone để hỗ trợ đội thi đánh giá kết quả.

### Cài đặt ban đầu

Cài đặt fiftyone

In [ ]:
! pip install fiftyone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Trong hướng dẫn này, chúng ta sẽ sử dụng CLIP và pytorch

In [ ]:
! pip install torch
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-r1zikxcg
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-r1zikxcg


Load dữ liệu keyframe từ thư mục chứa keyframe. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một Sample. Tất cả các Sample được lưu trong Dataset.

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import fiftyone as fo
import clip
import torch
import json
import numpy as np
from glob import glob
import os

dataset = fo.Dataset.from_images_dir('/gdrive/MyDrive/AI-HCM/Keyframes/KeyFramesC00_V00', name=None, tags=None, recursive=True)

 100% |█████████████| 26762/26762 [7.7s elapsed, 0s remaining, 4.8K samples/s]       


INFO:eta.core.utils: 100% |█████████████| 26762/26762 [7.7s elapsed, 0s remaining, 4.8K samples/s]       


In [ ]:
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


INFO:fiftyone.core.session.session:Session launched. Run `session.show()` to open the App in a cell output.


In [ ]:
!ls /gdrive/MyDrive/AI-HCM/Keyframes/KeyFramesC00_V00

C00_V0000  C00_V0015  C00_V0030  C00_V0045  C00_V0060  C00_V0075  C00_V0090
C00_V0001  C00_V0016  C00_V0031  C00_V0046  C00_V0061  C00_V0076  C00_V0091
C00_V0002  C00_V0017  C00_V0032  C00_V0047  C00_V0062  C00_V0077  C00_V0092
C00_V0003  C00_V0018  C00_V0033  C00_V0048  C00_V0063  C00_V0078  C00_V0093
C00_V0004  C00_V0019  C00_V0034  C00_V0049  C00_V0064  C00_V0079  C00_V0094
C00_V0005  C00_V0020  C00_V0035  C00_V0050  C00_V0065  C00_V0080  C00_V0095
C00_V0006  C00_V0021  C00_V0036  C00_V0051  C00_V0066  C00_V0081  C00_V0096
C00_V0007  C00_V0022  C00_V0037  C00_V0052  C00_V0067  C00_V0082  C00_V0097
C00_V0008  C00_V0023  C00_V0038  C00_V0053  C00_V0068  C00_V0083  C00_V0098
C00_V0009  C00_V0024  C00_V0039  C00_V0054  C00_V0069  C00_V0084  C00_V0099
C00_V0010  C00_V0025  C00_V0040  C00_V0055  C00_V0070  C00_V0085
C00_V0011  C00_V0026  C00_V0041  C00_V0056  C00_V0071  C00_V0086
C00_V0012  C00_V0027  C00_V0042  C00_V0057  C00_V0072  C00_V0087
C00_V0013  C00_V0028  C00_V0043  C00_V0058  C

Trích xuất thêm thông tin tên của video và frameid

In [ ]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('/', 2)
    sample.save()

In [ ]:
print(dataset.first())

<Sample: {
    'id': '6352e1e4b1ce0a6ab0b88ebf',
    'media_type': 'image',
    'filepath': '/gdrive/.shortcut-targets-by-id/1xsCSYNX8m-R6YuUExgx0G3vtWAmcC6s1/AI-HCM/Keyframes/KeyFramesC00_V00/C00_V0000/000000.jpg',
    'tags': [],
    'metadata': None,
    'video': 'C00_V0000',
    'frameid': '000000',
}>


In [ ]:
import json

Thêm thông tin kết quả của object detection.

In [ ]:
for sample in dataset:
    object_path = '/gdrive/MyDrive/AI-HCM/Objects/ObjectsC00_V00/{}.json'.format(sample['filepath'][-20:-4])
    with open(object_path) as jsonfile:
        det_data = json.load(jsonfile)
    detections = []
    for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
        # Convert to [top-left-x, top-left-y, width, height]
        boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
        scoref = float(score)
        
        # Only add objects with confidence > 0.4
        if scoref > 0.4:
            detections.append(
                fo.Detection(
                    label=cls,
                    bounding_box= boxf,
                    confidence=float(score)
                )
            )
    sample["object_faster_rcnn"] = fo.Detections(detections=detections)
    sample.save()
                           

Thêm thông tin CLIP embedding.

In [ ]:
all_keyframe = glob('/gdrive/MyDrive/AI-HCM/Keyframes/KeyFramesC00_V00/*/*.jpg')
video_keyframe_dict = {}
all_video = glob('/gdrive/MyDrive/AI-HCM/Keyframes/KeyFramesC00_V00/*')  
all_video = [v.rsplit('/',1)[-1] for v in all_video]

In [ ]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('/',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

In [ ]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

In [ ]:
for sample in dataset:
    object_path = '/gdrive/MyDrive/AI-HCM/CLIP_features/CLIPFeatures_S_C00_V00/{}.npy'.format(sample['filepath'][-20:-4])
    clip_embedding = np.load(object_path)
    sample['clip_embedding'] = clip_embedding
    sample.save()


In [ ]:
model, preprocess = clip.load("ViT-B/16", device="cpu")
model.eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

In [ ]:
text_search = "A female is drawing a landscape picture"

In [ ]:
text_tokens = clip.tokenize([text_search])

In [ ]:
with torch.no_grad():
    text_features = model.encode_text(text_tokens).float()

In [ ]:
for sample in dataset:
    a = sample['clip_embedding']
    query_similarity = (text_features @ a.reshape(1,512).T).cpu().numpy().item()
    sample['text_query_similarity'] = query_similarity
    sample.save()
    

In [ ]:
session.show()